# Part 2 Solution - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - Some of the options require a Mapbox API token:
    - However, we will be using the options that DO NOT require a token.
        - https://studio.mapbox.com/

# Loading Data from Part 1

In [ ]:
## Plotly is not included in your dojo-env
!pip install plotly

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [ ]:
## Load in csv.gz
df = pd.read_csv('Data/Fresno-wings.csv.gz')
df.head()

## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- 2. We also want to be able to show the restaurants:
    - name
    - price
    - type of transactions (pickup/delivery)
    - address

### Separating Latitude and Longitude

In [ ]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

- Why didn't that work???

In [ ]:
## slice out a single test coordinate
test_coord = df.loc[1, 'coordinates']
test_coord

In [ ]:
type(test_coord)

- Its not a dictionary anymore!!! What??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our `string dictionary` into an `actual dictionary`. 

In [ ]:
## Use json.loads on the test coordinate
# json.loads(test_coord)

### JSON requires double quotes!
We got a `JSON Decode Error` because JSON was `expecting double quotes` inside
of the dictionary

In [ ]:
# Check the single record
test_coord

### We are now going to use the .replace( ) function to replace single ' with double "

In [ ]:
## replace single ' with double " 
test_coord = test_coord.replace("'", '"')
test_coord

In [ ]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

In [ ]:
# viewing type after using json.loads
type(json.loads(test_coord))

### Now, how can we apply this same process to the entire column??

In [ ]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [ ]:
## slice out a single test coordinate
test_coord = df.loc[5, 'coordinates']
test_coord

In [ ]:
type(test_coord)

### Using .apply with pd.Series to convert a dictionary column into multiple columns
This is the process of unpacking the dictionary to columns

In [ ]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

In [ ]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df, df['coordinates'].apply(pd.Series)], axis = 1)
#df = df.drop(columns = 'coordinates')
df.head(2)

In [ ]:
df = df.drop(columns = 'coordinates')
df.head(2)

### Activity -Padlet : Unpacking dictionary


- https://padlet.com/swhaley9/unpacking-dictionaries-n19x0tiyn980jr0o

## Creating a Simple Map

- Mapbox API: https://www.mapbox.com/
- Mapbox API Documentation: https://docs.mapbox.com/api/overview/

- Use the plotly express `scatter_mapbox` function

In [ ]:
## use scatter_mapbox for map
px.scatter_mapbox(df, lat = 'latitude', lon = 'longitude', mapbox_style = 'open-street-map')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - rating
    - transaction type (delivery/takeout)
    - address
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df, lat = 'latitude', lon = 'longitude', mapbox_style = 'open-street-map', 
                 hover_name = 'name', hover_data = ['price', 'rating', 'transactions', 'location'])

### Fixing the Location Column

In [ ]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

> Also a string-dictionary...

In [ ]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df.head()

In [ ]:
## apply json.loads
df['location'] = df['location'].apply(json.loads)

# > Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [ ]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

In [ ]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [ ]:
## test applying our function
df['location'].apply(fix_address)

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [ ]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)

In [ ]:
## filter for businesses with display_location == "ERROR"
errors = df[df['display_location'] == 'Error']
errors

In [ ]:
## slice out a new test address and inspect
test_addr

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with " ' " in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [ ]:
## remove any rows where display_location == 'ERROR'
df = df.loc[df['display_location'] != 'Error']
df.head()

- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## slice out a new test address and inspect
test_addr = df.loc[7, 'display_location']['display_address']
test_addr

In [ ]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x: x['display_address'])
df.head(2)

- Almost done! We want to convert display_address to a string instead of a list of strings.
- We can use the string method .join to do so!

In [ ]:
## slice out a test_address
test_add = df.loc[7, 'display_address']
test_add

In [ ]:
## test using .join with a "\n"
'\n'.join(test_add)
print('\n'.join(test_add))

In [ ]:
## apply the join to every row with a lambda
df['address'] = df['display_address'].apply(lambda x: '\n'.join(x))
df.head()

In [ ]:
test_add = df.loc[7, 'address']
test_add

### Lastly, Fixing Transactions Column

In [ ]:
# Looking at a test transaction
test_transaction = df.loc[7, 'transactions']
test_transaction

In [ ]:
type(test_transaction)

- This is also a string and needs to be converted to a list.

In [ ]:
# Replacing single ' with double "
saved_test = test_transaction.replace("'", '"')
saved_test

In [ ]:
# Using json.loads on saved_test
json.loads(saved_test)

In [ ]:
# Applying transformations to entire column

# Create a new column where the single quotes are replaced with double quotes
df['transactions_split'] = df['transactions'].str.replace("'", '"')
# Apply json.loads to entire column
df['transactions_split'] = df['transactions_split'].apply(json.loads)
df['transactions_split'].head()

In [ ]:
df['transactions_split'].value_counts()

In [ ]:
# Converting transactions column into a one-hot-encoded column
exploded = df.explode('transactions_split')
exploded[['name', 'transactions', 'transactions_split']].head()

In [ ]:
# remove NaNs and find unique values
cols_to_make = exploded['transactions_split'].dropna().unique()
cols_to_make

In [ ]:
# Using a for loop with .str.contains to create new columns
for col in cols_to_make:
    df[col] = df['transactions'].str.contains(col)
df.head()

### Final Map

In [ ]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map', hover_name = 'name',
                         hover_data = ['price', 'rating', 'address', 'pickup', 
                                      'delivery'],
                         color = 'rating',
                         title = 'Restaurants with Wings in Fresno, TX')

pfig.show()

### Saving Final Figure

In [ ]:
## use fig.write_html to save map
pfig.write_html('final_map.html')